<a href="https://colab.research.google.com/github/K12-Analytics-Engineering/edfi-client-sdks/blob/main/Ed_Fi_142_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![logo](https://storage.googleapis.com/data-studio-codelab/academy_logo.png)

Welcome! This notebook is designed to be used with Ed-Fi 142, a course on the Ed-Fi Academy. Ed-Fi 142 focuses on pushing data from a SIS to the `/schools` endpoint. This Notebook looks at two methods for retrieving data from an Ed-Fi API as means of exploring what's inside the ODS.

It is recommended that you run each cell individually, reading the text and comments along the way.

In [ ]:
# run cell to clone edfi api sdk into environment
! git clone https://github.com/K12-Analytics-Engineering/edfi-client-sdks.git;
! mv /content/edfi-client-sdks/v5.3 /content/swagger_client;

# Imports and API credentials

This section will import the necessary python libraries as well as have you enter your Ed-Fi API credentials. It is recommended to use the same API credentials used in the SIS.

In [ ]:
import datetime
import os
import urllib3
import requests

from getpass import getpass
import pandas as pd

from google.colab import data_table
data_table.enable_dataframe_formatter()

import swagger_client

In [ ]:
# update base url value, and run cell to store api key and secret
EDFI_BASE_URL="https://api.ed-fi.org/v5.3/api"
EDFI_API_KEY = input("Enter your API key")
EDFI_API_SECRET = getpass("Enter your API secret")

# Retrieving an access token

Before you make API requests, you need to send your API key and secret to the `/oauth/token` endpoint to request an access token. Access tokens are used in the actual API requests and typically expire after 30 minutes. Once expired, API clients will request a new access token.

In [ ]:
# run cell to create post request to retrieve access token
access_response = requests.post(
    f"{EDFI_BASE_URL}/oauth/token",
    headers={ "Authorization": urllib3.util.make_headers(basic_auth=f'{EDFI_API_KEY}:{EDFI_API_SECRET}').get("authorization") },
    data={ "grant_type": "client_credentials" }
)

if access_response.ok:
    access_token_json = access_response.json()
    print(f"Token will expire in {access_token_json['expires_in'] / 60} minutes")
    print(f"Access token: {access_token_json['access_token']}")
else:
    print(f"Failed to retrieve access token with error {access_response.status_code}")

# Schools

Let's retrieve some data from the `/schools` endpoint using the client SDK.

Now that you have an access token, you can configure the Ed-Fi API client. This is using a Client SDK generated from an [OpenAPI Specification document](https://api.ed-fi.org/v5.3/api/metadata/data/v3/resources/swagger.json) released by the Ed-Fi Alliance.

In [ ]:
# initialize edfi client
edfi_client = swagger_client.ApiClient()
edfi_client.configuration.access_token = access_token_json['access_token']
edfi_client.configuration.host = f"{EDFI_BASE_URL}/data/v3"
edfi_client.configuration.username = EDFI_API_KEY
edfi_client.configuration.password = EDFI_API_SECRET

In [ ]:
# run cell to initialize SchoolApi class and call the get_schools() function
schools_api = swagger_client.SchoolsApi(api_client=edfi_client)
schools = schools_api.get_schools()

f'Retrieved {len(schools)} schools.'

`schools` is a List of EdFiSchool objects. This means we can loop through them and access various attributes.

In [ ]:
print("School ID, Name of institution")
for school in schools:
  print(f"{school.school_id}, {school.name_of_institution}")

Let's imagine you did not want to get all schools, but rather wanted to see if school id 255901001 was in the ODS.

In [ ]:
schools_api = swagger_client.SchoolsApi(api_client=edfi_client)
schools = schools_api.get_schools(
    school_id="255901001"
)

schools[0].name_of_institution

# Schools (alternative)

Let's look at schools again, but without using the client SDK. Here we are using the requests library to make API calls.

In [ ]:
endpoint = f'{EDFI_BASE_URL}/data/v3/ed-fi/schools?limit=100'
headers = {'Authorization': f'Bearer {edfi_client.configuration.access_token}'}
response = requests.get(endpoint, headers=headers)

f'Retrieved {len(response.json())} schools.'

If we wanted to see if school id 255901001 was in the ODS, we would include that in the URL parameters.

In [ ]:
endpoint = f'{EDFI_BASE_URL}/data/v3/ed-fi/schools?schoolId=255901001&limit=100'
headers = {'Authorization': f'Bearer {edfi_client.configuration.access_token}'}
school_response = requests.get(endpoint, headers=headers)

school_response.json()

In [ ]:
pd.json_normalize(response.json())

In [ ]:
df = pd.json_normalize(
    response.json(),
    record_path=["gradeLevels"],
    meta=["schoolId", "nameOfInstitution"])

df[["schoolId", "nameOfInstitution", "gradeLevelDescriptor"]].sort_values(by=['schoolId'])